In [1]:
%pip install --upgrade baolib


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import tempfile
from pathlib import Path

from baolib.bao import (
    Access,
    AccessChange,
    Vault,
    DB,
    Store,
    Replica,
    Message,
    Mailbox,
    local_store,
    set_bao_log_level,
    newKeyPair,
)

set_bao_log_level('info')

INFO  [04:33:05] (+71ms) core.syncTime[time.go:34] - clock offset 38.734579ms from time.google.com 


# Introduction
Bao is a secure storage library for sharing encrypted data. This notebook shows the Python binding (`baolib`) API.


## Identity
Create identities with `newKeyPair()` which returns a tuple of (publicID, privateID).

In [3]:
alice, alice_secret = newKeyPair()

print('Owner public ID:', alice)

keys = alice_secret.decode()
print(f'secp256k1 key: {keys["cryptKey"]}, ed25519 key: {keys["signKey"]}') 


Owner public ID: A3lgfatK56K9jdLuSBw8aJ0O3VO3rMBCi9AieJQOz7CUtEsrZprPVliqpeEx-B34Ozx5EYtzJmZAQ6y9NI0yG1I=
secp256k1 key: ufhPFx17MT4RFn4JIumM8eCQbT92o-pPcTGG-8-W1FU=, ed25519 key: 41eONBwJWX3GgZRuxovxlmEKExk3Vn4MakRyDYFlphc=


## Local database
Use a local SQLite database to track metadata. Here we start with a clean DB in a temp directory.


In [4]:
temp_root = Path(tempfile.gettempdir()) / 'bao_python_sample'
temp_root.mkdir(parents=True, exist_ok=True)

def new_db_path(name: str) -> DB:
    db_path = temp_root / name
    if db_path.exists():
        db_path.unlink()
    return str(db_path)

db_path = new_db_path('bao.db')
db = DB('sqlite3', db_path)
print('Database at', db_path)


Database at /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db


INFO  [04:33:07] (+0ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db with driver sqlite3


## Create a vault
A vault combines the database, an identity, and a storage URL. The storage can be local (`file://`), S3, SFTP, etc.


In [5]:
store = Store(local_store('vault', str(temp_root / 'vault')))
vault = Vault.create(Vault.users, alice_secret, store, db)
vault

INFO  [04:33:08] (+15ms) Successfully wiped data from store vault
INFO  [04:33:08] (+17ms) vault.(*Vault).convertToChanges[access.go:134] - successfully created 2 changes for domain users
INFO  [04:33:08] (+18ms) vault.(*Vault).SyncAccess[access.go:37] - staged ChangeAccess: userID=115bf834b9d18530, access=RWA in vault|users
INFO  [04:33:08] (+19ms) vault.(*Vault).SyncAccess[access.go:37] - staged AddKey: keyId=408102377566490624,  users=115bf834b9d18530  in vault|users
INFO  [04:33:08] (+20ms) vault.(*Vault).exportBlocksToStorage[blockchain.go:343] - Config: retention=0s, maxStorage=0, segmentInterval=0s, syncCooldown=0s, waitTimeout=0s, filesSyncPeriod=0s, cleanupPeriod=0s, blockChainSyncPeriod=0s, ioThrottle=0 by 115bf834b9d18530 in vault|users
INFO  [04:33:08] (+21ms) vault.(*ChangeAccess).Apply[changes.go:303] - my access for vault vault|users changed to RWA
INFO  [04:33:08] (+21ms) vault.(*Vault).exportBlocksToStorage[blockchain.go:343] - ChangeAccess: userID=115bf834b9d18530, ac

vault|users

## Access control
Grant permissions to groups. Built-in groups include `admins`, `users`, and `public`. Permissions are bitmasks (`Access.read`, `Access.write`, `Access.admin`).


In [6]:
bob, bob_secret = newKeyPair()
vault.sync_access([
    AccessChange(bob, Access.read),
])

accesses = vault.get_accesses()
for user, access in accesses.items():
    print(f"User {user} has access level: {access}")

print('Bob specific access:', vault.get_access(bob))

User A3lgfatK56K9jdLuSBw8aJ0O3VO3rMBCi9AieJQOz7CUtEsrZprPVliqpeEx-B34Ozx5EYtzJmZAQ6y9NI0yG1I= has access level: 7
User AsBuhTlsOGjy877Nd1Uz6imtnWWx2hp9FOU6dDkmgCdTzxTASeOfmLv3UF0RK7hEF-es0YVK2eB2Fl7qKOOBskE= has access level: 1
Bob specific access: 1


INFO  [04:33:10] (+0ms) vault.(*Vault).convertToChanges[access.go:134] - successfully created 2 changes for domain users
INFO  [04:33:10] (+1ms) vault.(*Vault).SyncAccess[access.go:37] - staged ActiveKeySet: user: 524aa3a51856128b, key ids: 5a9def33ed7d000,  in vault|users
INFO  [04:33:10] (+2ms) vault.(*Vault).SyncAccess[access.go:37] - staged ChangeAccess: userID=524aa3a51856128b, access=R in vault|users
INFO  [04:33:10] (+3ms) vault.(*Vault).exportBlocksToStorage[blockchain.go:343] - ActiveKeySet: user: 524aa3a51856128b, key ids: 5a9def33ed7d000,  by 115bf834b9d18530 in vault|users
INFO  [04:33:10] (+4ms) vault.(*Vault).exportBlocksToStorage[blockchain.go:343] - ChangeAccess: userID=524aa3a51856128b, access=R by 115bf834b9d18530 in vault|users


## Open an existing vault
Anyone with the vault URL, their private ID, and the creator's public ID can reopen the vault.


In [7]:
vault = Vault.open(Vault.users, alice_secret, alice, store, db)
vault

INFO  [04:33:11] (+1ms) vault.Open[open.go:88] - successfully opened vault vault|users


vault|users

## Files and data
Write and read files using group permissions.


In [8]:
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

vault.write('docs/hello.txt', src=str(data_path)) # write file to vault
ls = vault.read_dir('docs', None, 0, 10)                        # list files in docs/

print('Files in docs/:', [l['name'] if isinstance(l, dict) else l.name for l in ls])

Files in docs/: ['hello.txt']


In [9]:
out_path = temp_root / 'hello.out'
vault.read('docs/hello.txt', str(out_path))
print('written content:', out_path.read_text())


written content: Hello from Bao!


INFO  [04:33:13] (+3ms) vault.(*Vault).Read[read.go:120] - successfully read file docs/hello.txt to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.out in 3.456917ms


I/O operations can be asynchronous by using the option _async_operation_ or _scheduled_operation_

In [10]:
import time
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

file = vault.write('docs/hello.txt', src=str(data_path), options=Vault.async_operation) # write file to vault
start = time.perf_counter_ns()

# other operations can be done here while write is in progress
vault.wait_files([file.file_id])      # wait for write to complete
print(f'File written in {(time.perf_counter_ns() - start) / 1_000_000 } ms')

File written in 1.334666 ms


INFO  [04:33:15] (+1ms) vault.(*Vault).waitFile[wait_files.go:60] - File ID 2 completed I/O operation
INFO  [04:33:15] (+1ms) vault.(*Vault).WaitFiles[wait_files.go:36] - Successfully synchronized vault with store ID vault|users in 1.194125ms


In [11]:
file = vault.stat('docs/hello.txt')
print(f'File stats: name = {file.name}', f'size = {file.size} bytes', f'flags = {file.flags}', 
          f'author = {file.author_id}')

#vault.delete('docs/hello.txt')

File stats: name = docs/hello.txt size = 15 bytes flags = 0 author = A3lgfatK56K9jdLuSBw8aJ0O3VO3rMBCi9AieJQOz7CUtEsrZprPVliqpeEx-B34Ozx5EYtzJmZAQ6y9NI0yG1I=


INFO  [04:33:16] (+0ms) vault.(*Vault).Stat[stat.go:40] - successfully got file info for docs/hello.txt: id=2, modTime=2026-01-31 04:33:15.282 +0100 CET, size=15, allocated=167, isDir=false


## Exchange with another peer
Create a new database to simulate a different peer. Read the content.


In [12]:
#set_bao_log_level('info')

db2_path = new_db_path('bao2.db')
db2 = DB('sqlite3', db2_path)
vault2 = Vault.open(Vault.users, bob_secret, alice, store, db2)
ls = vault2.read_dir('docs', None, 0, 10)

print('Alice sees files in docs/:', [l.name for l in ls])

Alice sees files in docs/: ['hello.txt']


INFO  [04:33:19] (+0ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao2.db with driver sqlite3
INFO  [04:33:19] (+9ms) vault.(*Vault).importBlockFromStorage[blockchain.go:242] - applied change Config: retention=0s, maxStorage=0, segmentInterval=0s, syncCooldown=0s, waitTimeout=0s, filesSyncPeriod=0s, cleanupPeriod=0s, blockChainSyncPeriod=0s, ioThrottle=0 from block users/blockchain/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA author 115bf834b9d18530
INFO  [04:33:19] (+9ms) vault.(*Vault).importBlockFromStorage[blockchain.go:242] - applied change ChangeAccess: userID=115bf834b9d18530, access=RWA from block users/blockchain/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA author 115bf834b9d18530
INFO  [04:33:19] (+9ms) vault.(*Vault).importBlockFromStorage[blockchain.go:242] - applied change AddKey: keyId=408102377566490624,  users=1

In [13]:
vault2.read('docs/hello.txt', str(temp_root / 'hello.collab.out'))
(temp_root / 'hello.collab.out').read_text()

INFO  [04:33:20] (+3ms) vault.(*Vault).Read[read.go:120] - successfully read file docs/hello.txt to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.collab.out in 3.265916ms


'Hello from Bao!'

In [15]:
vault2.write('docs/hello.txt', src=str(data_path))
versions =vault2.versions('docs/hello.txt')

vault2.read(versions[-1].name, str(temp_root / 'hello.versioned.out'))
(temp_root / 'hello.versioned.out').read_text()

INFO  [04:34:06] (+0ms) vault.(*Vault).WaitFiles[wait_files.go:36] - Successfully synchronized vault with store ID vault|users in 30.708µs
INFO  [04:34:06] (+0ms) vault.(*Vault).Versions[versions.go:58] - successfully got file versions for hello.txt: 4
INFO  [04:34:06] (+1ms) vault.(*Vault).Read[read.go:120] - successfully read file docs/hello.txt:3 to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.versioned.out in 1.827041ms


'Hello from Bao!'

INFO  [05:03:05] (+1739212ms) core.syncTime[time.go:34] - clock offset 14.305609ms from time.google.com 
INFO  [05:33:05] (+3539225ms) core.syncTime[time.go:34] - clock offset 11.405685ms from time.google.com 
INFO  [06:03:05] (+5339250ms) core.syncTime[time.go:34] - clock offset 30.667482ms from time.google.com 
INFO  [06:33:05] (+7139320ms) core.syncTime[time.go:34] - clock offset 12.383112ms from time.google.com 
INFO  [07:28:01] (+8939224ms) core.syncTime[time.go:34] - clock offset 10.421456ms from time.google.com 
INFO  [09:18:50] (+10739221ms) core.syncTime[time.go:34] - clock offset 8.999673ms from time.google.com 
INFO  [09:50:28] (+12539222ms) core.syncTime[time.go:34] - clock offset 13.493494ms from time.google.com 
INFO  [10:20:28] (+14339407ms) core.syncTime[time.go:34] - clock offset 12.517736ms from time.google.com 
INFO  [10:50:28] (+16139282ms) core.syncTime[time.go:34] - clock offset 46.639947ms from time.google.com 
INFO  [11:20:28] (+17939219ms) core.syncTime[time.go

## Metadata
The write API supports metadata.

In [29]:
import json
vault2.write('docs/hello-attrs.txt', attrs=json.dumps({'edited_by': 'bob'}).encode('utf-8'))

file = vault2.stat('docs/hello-attrs.txt')
print('File attributes:', json.loads(file.attrs.decode('utf-8')))

File attributes: {'edited_by': 'bob'}


INFO  [04:25:45] (+0ms) vault.(*Vault).Stat[stat.go:40] - successfully got file info for docs/hello-attrs.txt: id=4, modTime=2026-01-31 04:25:45.684 +0100 CET, size=0, allocated=168, isDir=false


## Messaging
Send and receive `Message` objects inside a vault directory.


In [17]:
mailbox = Mailbox(vault, 'mailbox')
mailbox.send(Message('Greetings', 'Hello team!'))
mailbox.receive(0, 0)

[Message(subject='Greetings', body='Hello team!', attachments=[], fileInfo={'id': 3, 'name': '5a945043f1a7000', 'realm': 'users', 'size': 0, 'allocatedSize': 392, 'modTime': '2026-01-30T17:20:36.195+01:00', 'isDir': False, 'flags': 0, 'attrs': 'eyJzdWJqZWN0IjoiR3JlZXRpbmdzIiwiYm9keSI6IkhlbGxvIHRlYW0hIiwiYXR0YWNobWVudHMiOltdLCJmaWxlSW5mbyI6eyJpZCI6MCwibmFtZSI6IiIsInJlYWxtIjoiIiwic2l6ZSI6MCwiYWxsb2NhdGVkU2l6ZSI6MCwibW9kVGltZSI6IjAwMDEtMDEtMDFUMDA6MDA6MDBaIiwiaXNEaXIiOmZhbHNlLCJmbGFncyI6MCwia2V5SWQiOjAsInN0b3JlRGlyIjoiIiwic3RvcmVOYW1lIjoiIiwiYXV0aG9ySWQiOiIifX0=', 'keyId': 407932951282216960, 'storeDir': 'users/data/20260130000000', 'storeName': '6iywdzq8', 'authorId': 'A2bE9-oSjsPM04PaRqm2-kMepZo-pl3aWDbA5rjU_qq-mTDCBO4LUorBF_j7vyTxvQXl-uI9_4-SrwkPytl6txI='})]

## Replica interface
Attach a SQL interface that replicates statements through the vault. The interface allows placeholders for the SQL expressions.
The placeholders are defined in the statementes that creates the DB.
Each statements is introduced by a comment:
- a comment starting with _INIT_ and _version_ number is used during the creation of the DB
- other comments are placeholders for the queries

In [18]:
ddl = '''
-- INIT 1.0
CREATE TABLE IF NOT EXISTS notes (id INTEGER PRIMARY KEY, body TEXT);

-- INSERT_NOTE 1.0
INSERT INTO notes (id, body) VALUES (:id, :body);

-- SELECT_ALL_NOTES 1.0
SELECT * FROM notes;
'''

db_path = temp_root / 'data.db'
if db_path.exists():
    db_path.unlink()
    
data_db = DB('sqlite3', str(db_path), ddl=ddl)
replica = Replica(vault, data_db)
replica.exec('INSERT_NOTE', {'id': 1, 'body': 'hello'})
replica.exec('INSERT_NOTE', {'id': 2, 'body': 'bao'})
replica.fetch('SELECT_ALL_NOTES', {})

replica.sync()

INFO  [17:20:41] (+2ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/data.db with driver sqlite3


2

Another peer can receive the updates

In [19]:
db_path2 = temp_root / 'data2.db'
if db_path2.exists():
    db_path2.unlink()
data_db2 = DB('sqlite3', str(db_path2), ddl=ddl)
replica2 = Replica(vault2, data_db2)
replica2.sync() 
replica2.fetch('SELECT_ALL_NOTES', {})

INFO  [17:20:46] (+1ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/data2.db with driver sqlite3


[[1, 'hello'], [2, 'bao']]

INFO  [17:42:12] (+1286067ms) core.syncTime[time.go:34] - clock offset 13.260081ms from time.google.com 
